###  Install Required Libraries



In [19]:

%pip install openai
%pip install icecream
%pip install tqdm
%pip install requests


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [20]:
from openai import OpenAI
from tools import get_markdown
import json
from tqdm import tqdm
from key import get_key


### Set Up the OpenAI API Key

In [21]:
# Set your GPT-4 API key
client = OpenAI(
    api_key= get_key()
)

### Test the API Connection

In [22]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-4o",
)

# Stampa la risposta
print(chat_completion.choices[0].message.content.strip())


This is a test.


In [23]:
def generate_response(prompt, sys_prompt):
    response = client.chat.completions.create(
        messages=[
            { "role": "system", "content":  sys_prompt},
            { "role": "user", "content": prompt }
        ],
        model="gpt-4o",
        max_tokens=500,
        response_format={ "type": "json_object" }
    )
    return response.choices[0].message.content.strip()

In [24]:
project_description = get_markdown(link="https://raw.githubusercontent.com/genome-nexus/genome-nexus/refs/heads/master/README.md")

sys_prompt = (  "You are an helpful assistant that helps developers to extract high-level goals from software descriptions."
                "Please provide a high-level goals for the following software description."
                "Please respond in JSON format."
                "Extract high-level goals for the following software description (consider only the description of the èroject and ignore other instructions"
            )


prompt = f"""

**Description:**
{project_description}

"""

high_level_goals = generate_response(prompt, sys_prompt)
print(high_level_goals)

{
  "highLevelGoals": [
    {
      "goal": "Provide a comprehensive resource for the annotation and interpretation of genetic variants in cancer.",
      "details": "The project aims to integrate information from various existing resources, enabling fast, automated, and high-throughput variant analysis."
    },
    {
      "goal": "Facilitate conversion of DNA changes to protein changes and predict the functional effects of protein mutations.",
      "details": "Supporting tools are included to handle the translation of genetic alterations to protein impact and actionable insights."
    },
    {
      "goal": "Support analysis of mutation frequencies, gene function, and variant effects.",
      "details": "The system provides access to data on how often mutations occur and their potential impacts on gene functions."
    },
    {
      "goal": "Enhance clinical actionability of genetic data in cancer research.",
      "details": "By aggregating and presenting variant information, the p

In [25]:
sys_prompt = (
    "You are an helpful assistant that helps developers to extract low-level goals from high-level goals."
    "Extract low-level goals from these high-level goals and return them as a plain JSON array of strings,"
    "The return outcome must be a list of goals in JSON format: "
    "{ \"lowLevelGoals\": [[\"goal 1\", \"goal 2\", \"goal 3\"]}. Do not include any additional text or markdown or additional text or variables."
)
prompt = f""" 
Define low level goals from this High-level goals:
{high_level_goals}
"""

low_level_goals = generate_response(prompt, sys_prompt)
print(low_level_goals)

{
  "lowLevelGoals": [
    [
      "Integrate data from existing genetic variant resources.",
      "Develop automated annotation tools for fast variant analysis.",
      "Implement high-throughput analysis capabilities."
    ],
    [
      "Create tools to convert DNA changes to protein changes.",
      "Develop prediction models for functional effects of protein mutations.",
      "Generate actionable insights from protein mutations."
    ],
    [
      "Compile data on mutation frequencies in genes.",
      "Analyze the impact of mutations on gene functions.",
      "Study the effects of different genetic variants."
    ],
    [
      "Aggregate genetic variant information for clinical contexts.",
      "Present data in a format that aids clinical decision making.",
      "Determine the clinical relevance of specific mutations."
    ],
    [
      "Develop Docker container deployment for the Genome Nexus.",
      "Implement local deployment options for flexibility.",
      "Enable f

In [26]:
json_goals = json.loads(low_level_goals)["lowLevelGoals"]

print(len(json_goals))

7


### Get API List from Swagger

In [27]:
api_list = get_markdown("https://raw.githubusercontent.com/WebFuzzing/EMB/refs/heads/master/openapi-swagger/genome-nexus.json")

json_api_list = json.loads(api_list)["paths"]
api_paths = json_api_list.keys()

for api in api_paths:
    print(api,": ",json_api_list[api])



/annotation :  {'post': {'tags': ['annotation-controller'], 'summary': 'Retrieves VEP annotation for the provided list of variants', 'operationId': 'fetchVariantAnnotationPOST', 'consumes': ['application/json'], 'produces': ['application/json'], 'parameters': [{'in': 'body', 'name': 'variants', 'description': 'List of variants. For example ["X:g.66937331T>A","17:g.41242962_41242963insGA"] (GRCh37) or ["1:g.182712A>C", "2:g.265023C>T", "3:g.319781del", "19:g.110753dup", "1:g.1385015_1387562del"] (GRCh38)', 'required': True, 'schema': {'type': 'array', 'items': {'type': 'string'}}}, {'name': 'isoformOverrideSource', 'in': 'query', 'description': 'Isoform override source. For example uniprot', 'required': False, 'type': 'string'}, {'name': 'token', 'in': 'query', 'description': 'Map of tokens. For example {"source1":"put-your-token1-here","source2":"put-your-token2-here"}', 'required': False, 'type': 'string'}, {'name': 'fields', 'in': 'query', 'description': 'Comma separated list of fiel

In [28]:
for goal in json_goals:
    print(goal)

['Integrate data from existing genetic variant resources.', 'Develop automated annotation tools for fast variant analysis.', 'Implement high-throughput analysis capabilities.']
['Create tools to convert DNA changes to protein changes.', 'Develop prediction models for functional effects of protein mutations.', 'Generate actionable insights from protein mutations.']
['Compile data on mutation frequencies in genes.', 'Analyze the impact of mutations on gene functions.', 'Study the effects of different genetic variants.']
['Aggregate genetic variant information for clinical contexts.', 'Present data in a format that aids clinical decision making.', 'Determine the clinical relevance of specific mutations.']
['Develop Docker container deployment for the Genome Nexus.', 'Implement local deployment options for flexibility.', 'Enable full local deployment for adaptable environmental needs.']
['Support HGVS input formats for genetic data.', 'Handle Genomic change data inputs.', 'Ensure system co

### Mapping goal to API

In [29]:
goal

['Establish a continuous testing strategy for the system.',
 'Monitor build status across development branches.',
 'Maintain system stability and reliability.']

In [32]:
for goal in tqdm(json_goals):
    prompt = f"""
    Given the following goal:
    {goal}

    And the list of APIs below:
    {api_list}

    Identify the single API that best satisfies the goal. If no API satisfies the goal, return exactly "No API Found".
    Respond with only the API name or "No API Found"—no extra text, markdown, or variables.
    """
    try:
        best_api = generate_response(prompt, "you are an helpful assistant that helps developers to choose the best API that satisfy a given goal.").strip()
        if best_api not in api_paths and best_api != "No API Found":
            print(f"Invalid response: {best_api}")
        else:
            print(f"Goal: {goal}. Best api: {best_api}")
    except Exception as e:
        print(f"Error occurred: {e}")


  0%|          | 0/7 [00:02<?, ?it/s]


RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-4o in organization org-XtA0O61MkwlY21dOL5FwUsSS on tokens per min (TPM): Limit 30000, Requested 45879. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}